In [5]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
import torch.utils.data as data
from utils import *
from models import *
import numpy as np

## Data Setup

In [6]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
]) 

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

dataloader = datasets.CIFAR10
batch_size = 64
test_bs = 256

trainset = dataloader(root='./data', train=True, download=True, transform=transform_train)
trainloader = data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = dataloader(root='./data', train=False, download=True,
        transform=transform_test)
testloader = data.DataLoader(testset, batch_size=test_bs, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Model Init

In [50]:
model = AlexNet(num_classes=10).to(device)

## Train

In [51]:
EPOCHS = 50
lr = 1e-3
optim = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()
lambda_TV = 1e-4
layer_mask = [0]

### Without TV:

In [52]:
def eval_model(model, testloader):
    model.eval()
    class_losses = []
    accs = []
    with torch.no_grad():
        for batch_input, batch_labels in testloader:
            batch_input = batch_input.to(device)
            batch_labels = batch_labels.to(device)
            
            batch_output = model(batch_input)
            batch_pred = torch.argmax(batch_output, dim = 1)
            acc = torch.mean(torch.eq(batch_pred,batch_labels).type(torch.float))
            class_loss = criterion(batch_output,batch_labels)
            
            accs.append(acc.item())
            class_losses.append(class_loss.item())
    return np.mean(accs),np.mean(class_losses)

In [53]:
for e in range(EPOCHS):
    # validation
    val_acc, val_loss = eval_model(model,testloader)
    print("Epoch", e, "| Test Acc:", "{:8.4f}".format(val_acc), "| Test Loss:", "{:8.4f}".format(val_loss), end = "")
    
    # training
    model.train()
    class_losses = []
    TV_losses = []
    ok = 0
    with torch.no_grad():
        init = TVLossMat(model, layer_mask).item()
    for batch_input, batch_labels in trainloader:
        optim.zero_grad()
        batch_input = batch_input.to(device)
        batch_labels = batch_labels.to(device)
        batch_output = model(batch_input)
        
        class_loss = criterion(batch_output,batch_labels)
        loss = class_loss
        loss.backward()
        optim.step()
        
        class_losses.append(class_loss.item())
    print(" | Train loss:", "{:8.4f}".format(np.mean(class_losses)), "| Init TV:", "{:8.4f}".format(init))

Epoch 0 | Test Acc:   0.1021 | Test Loss:   2.3026 | Train loss:   1.8410 | Init TV: 1043.0659
Epoch 1 | Test Acc:   0.3868 | Test Loss:   1.6481 | Train loss:   1.5636 | Init TV: 1104.4001
Epoch 2 | Test Acc:   0.4708 | Test Loss:   1.4891 | Train loss:   1.4197 | Init TV: 1174.3119
Epoch 3 | Test Acc:   0.5138 | Test Loss:   1.3320 | Train loss:   1.3369 | Init TV: 1248.3767
Epoch 4 | Test Acc:   0.5326 | Test Loss:   1.2915 | Train loss:   1.2899 | Init TV: 1316.1458
Epoch 5 | Test Acc:   0.5582 | Test Loss:   1.2257 | Train loss:   1.2471 | Init TV: 1384.0524
Epoch 6 | Test Acc:   0.5603 | Test Loss:   1.2287 | Train loss:   1.2112 | Init TV: 1450.5839
Epoch 7 | Test Acc:   0.5486 | Test Loss:   1.2437 | Train loss:   1.1870 | Init TV: 1515.5562
Epoch 8 | Test Acc:   0.5805 | Test Loss:   1.2078 | Train loss:   1.1562 | Init TV: 1577.9421
Epoch 9 | Test Acc:   0.5949 | Test Loss:   1.1597 | Train loss:   1.1299 | Init TV: 1634.8892
Epoch 10 | Test Acc:   0.5806 | Test Loss:   1.183

KeyboardInterrupt: 

### With TV

In [54]:
model = AlexNet(num_classes=10).to(device)
EPOCHS = 25
lr = 1e-3
optim = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()
lambda_TV = 1e-4
layer_mask = [0]

In [55]:
for e in range(EPOCHS):
    # validation
    val_acc, val_loss = eval_model(model,testloader)
    print("Epoch", e, "| Test Acc:", "{:8.4f}".format(val_acc), "| Test Loss:", "{:8.4f}".format(val_loss), end = "")
    
    # training
    model.train()
    class_losses = []
    TV_losses = []
    with torch.no_grad():
        init = TVLossMat(model, layer_mask).item()
    for batch_input, batch_labels in trainloader:
        optim.zero_grad()
        batch_input = batch_input.to(device)
        batch_labels = batch_labels.to(device)
        batch_output = model(batch_input)
        
        class_loss = criterion(batch_output,batch_labels)
        TV_loss = TVLossMat(model, layer_mask)
        loss = class_loss + lambda_TV*TV_loss
        loss.backward()
        optim.step()
        
        class_losses.append(class_loss.item())
        TV_losses.append(TV_loss.item()*lambda_TV)
    
    print(" | Train loss:", "{:8.4f}".format(np.mean(class_losses)), "| Init TV:", "{:8.4f}".format(init), "| TV loss:", "{:8.4f}".format(np.mean(TV_losses)))

Epoch 0 | Test Acc:   0.0977 | Test Loss:   2.3027 | Train loss:   1.8552 | Init TV: 1039.3700 | TV loss:   0.0365
Epoch 1 | Test Acc:   0.3759 | Test Loss:   1.5947 | Train loss:   1.5559 | Init TV: 282.5186 | TV loss:   0.0296
Epoch 2 | Test Acc:   0.4462 | Test Loss:   1.5486 | Train loss:   1.4050 | Init TV: 314.3334 | TV loss:   0.0338
Epoch 3 | Test Acc:   0.5090 | Test Loss:   1.3243 | Train loss:   1.3135 | Init TV: 359.6890 | TV loss:   0.0377
Epoch 4 | Test Acc:   0.5545 | Test Loss:   1.2513 | Train loss:   1.2425 | Init TV: 393.9750 | TV loss:   0.0410
Epoch 5 | Test Acc:   0.5740 | Test Loss:   1.2051 | Train loss:   1.1993 | Init TV: 428.5070 | TV loss:   0.0440
Epoch 6 | Test Acc:   0.5754 | Test Loss:   1.1829 | Train loss:   1.1580 | Init TV: 449.0258 | TV loss:   0.0464
Epoch 7 | Test Acc:   0.5953 | Test Loss:   1.1479 | Train loss:   1.1213 | Init TV: 470.5056 | TV loss:   0.0485
Epoch 8 | Test Acc:   0.6149 | Test Loss:   1.1060 | Train loss:   1.0957 | Init TV: 49